In [2]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import pickle

SCORE_DIR = "data/scores"
box_scores = os.listdir(SCORE_DIR)
len(box_scores)

box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

def parse_html(box_score):
    with open(box_score, 'r', encoding='utf-8', errors='ignore') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [4]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]
    return line_score

In [5]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [6]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [7]:
base_cols = None
games = []
games2 = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        df = pd.read_html(str(soup), attrs = {"id": f"box-{team}-game-basic"}, index_col=0)[0]
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1, :]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1, :].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep = "first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        summary = summary[base_cols]
        summaries.append(summary)

    summary = pd.concat(summaries, axis = 1).T

    game = pd.concat([summary, line_score], axis = 1)
    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis = 1)
    full_game["season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format = "%Y%m%d")
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 50 == 0:
        print(f"{len(games)} / {len(box_scores)}")

50 / 4943
100 / 4943
150 / 4943
200 / 4943
250 / 4943
300 / 4943
350 / 4943
400 / 4943
450 / 4943
500 / 4943
550 / 4943
600 / 4943
650 / 4943
700 / 4943
750 / 4943
800 / 4943
850 / 4943
900 / 4943
950 / 4943
1000 / 4943
1050 / 4943
1100 / 4943
1150 / 4943
1200 / 4943
1250 / 4943
1300 / 4943
1350 / 4943
1400 / 4943
1450 / 4943
1500 / 4943
1550 / 4943
1600 / 4943
1650 / 4943
1700 / 4943
1750 / 4943
1800 / 4943
1850 / 4943
1900 / 4943
1950 / 4943
2000 / 4943
2050 / 4943
2100 / 4943
2150 / 4943
2200 / 4943
2250 / 4943
2300 / 4943
2350 / 4943
2400 / 4943
2450 / 4943
2500 / 4943
2550 / 4943
2600 / 4943
2650 / 4943
2700 / 4943
2750 / 4943
2800 / 4943
2850 / 4943
2900 / 4943
2950 / 4943
3000 / 4943
3050 / 4943
3100 / 4943
3150 / 4943
3200 / 4943
3250 / 4943
3300 / 4943
3350 / 4943
3400 / 4943
3450 / 4943
3500 / 4943
3550 / 4943
3600 / 4943
3650 / 4943
3700 / 4943
3750 / 4943
3800 / 4943
3850 / 4943
3900 / 4943
3950 / 4943
4000 / 4943
4050 / 4943
4100 / 4943
4150 / 4943
4200 / 4943
4250 / 4943


In [8]:
len(games)

4943

In [12]:
games_df.to_csv("nba_games.csv")